In [1]:
import umap
import matplotlib.pyplot as plt
import numpy as np
from custom_umap import CustomUMAP
import torch
from transformers import AutoModel, AutoTokenizer

/Users/mcevans/WebDev/PROJECTS/headspace/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
custom_umap = CustomUMAP()

In [3]:

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
sample_text = [

    "Restoring natural habitats helps maintain ecological balance.",
    "Restoring natural habitats helps maintain earth balance.",
    "Restoring natural habitats helps maintain water balance.",
    "Restoring natural habitats helps maintain heat balance.",
    "asdf ass gdasga.", 
    "asd fass gdasga.", 
    "asdf assgda sga.", 
    "asdf   assgdasga.",
]

inputs = [tokenizer(text, return_tensors="pt",
                    truncation=True, max_length=512) for text in sample_text]
outputs = []
for i in inputs:
    with torch.no_grad():
        outputs.append(model(**i))

# Assuming you want the last hidden states (token-level embeddings)
embeddings = [output[0] for output in outputs]

In [4]:

# Convert the list of tensors to a NumPy array
embeddings_np = custom_umap.to_numpy(embeddings)

In [5]:

reducer = umap.UMAP(
    n_components=2, n_neighbors=3, low_memory=True)
# Averaging the token embeddings to get a single vector per sentence
embedding_average = [torch.mean(embedding.squeeze(), dim=0)
                        for embedding in embeddings]

# Convert the list of tensors to a NumPy array
embedding_np = torch.stack(
    embedding_average).detach().numpy()


out = reducer.fit_transform(embeddings)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (8, 1) + inhomogeneous part.

In [ ]:
# 2D Visualization
plt.scatter(reduced_embeddings[:, 0], reduced_embeddings[:, 1])

# Adding labels and title for clarity
plt.xlabel('UMAP Dimension 1')
plt.ylabel('UMAP Dimension 2')
plt.title('2D UMAP Projection of BERT Sentence Embeddings')

# Show the plot
plt.show()